In [65]:
from arcgis.features import GeoAccessor
from arcgis.geometry import Geometry
import pandas as pd

import sys
sys.path.append('../../src')
import geoai_retail
from ba_data_paths import ba_data

In [66]:
dest_fc = r'../../src/tests/test_data.gdb/coffee_pdx'
dest_id_fld = 'LOCNUM'
new_loc = Geometry({'x': -122.7020867, 'y': 45.4399585, 'spatialReference': {'wkid': 4326}})
n_th_locations = 4

## Get Existing Locations

In [51]:
dest_df = GeoAccessor.from_featureclass(dest_fc)
dest_df.sample(5)

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,SALESVOL,HDBRCH,ULTNUM,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,open_status,SHAPE
18,19,427528962,SYMPOSIUM COFFEE,SW MAIN ST,PORTLAND,OR,Oregon,97223,6242,72251505,581228,346.0,,000000000,6.0,,,2,PointAddress,M,100.0000,INFOGROUP,0,None,"{""x"": -122.7713999998274, ""y"": 45.430800000318..."
71,72,415522522,HUMAN BEAN,SW BEAVERTON HILLSDALE HWY,PORTLAND,OR,Oregon,97225,2215,72251505,581228,288.0,2,509729190,5.0,,,2,StreetAddress,M,90.9063,INFOGROUP,0,None,"{""x"": -122.76259999983552, ""y"": 45.48630000008..."
90,91,523617983,STARBUCKS,SW BARNES RD,PORTLAND,OR,Oregon,97225,6200,72251505,581228,748.0,2,303651459,13.0,4,,3,PointAddress,M,100.0000,INFOGROUP,0,None,"{""x"": -122.75470000011552, ""y"": 45.51559999960..."
92,93,724622018,STARBUCKS,SW BARNES RD,PORTLAND,OR,Oregon,97225,6203,72251505,581228,863.0,2,303651459,15.0,4,,3,PointAddress,M,100.0000,INFOGROUP,0,None,"{""x"": -122.75470000011552, ""y"": 45.51559999960..."
3,4,438669357,DA MIMMO,SW RAILROAD ST,SHERWOOD,OR,Oregon,97140,,72251505,581228,231.0,,000000000,4.0,,,1,PointAddress,M,100.0000,INFOGROUP,0,None,"{""x"": -122.84099999985409, ""y"": 45.35620000011..."


In [76]:
webmap = dest_df.spatial.plot()
webmap.basemap = 'dark-gray-vector'
webmap

MapView(layout=Layout(height='400px', width='100%'))

## Find Nearest Existing N-th Locations to New Location

In [77]:
symbol = {"angle":0,"xoffset":12,"yoffset":12,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/Basic/GreenFlag.png","contentType":"image/png","width":24,"height":24}
new_loc_id = dest_df[dest_id_fld].astype('int64').max() + 100

new_loc_df = pd.DataFrame([(new_loc_id, new_loc)], columns=['ID', 'SHAPE'])
new_loc_df.spatial.set_geometry('SHAPE')
new_loc_df.spatial.plot(map_widget=webmap, symbol=symbol)

True

In [78]:
nearest_existing_df = geoai_retail.proximity_local.get_closest_solution(new_loc_df, 'ID', dest_df, dest_id_fld, network_dataset=ba_data.usa_network_dataset, destination_count=n_th_locations)
nearest_existing_df

,origin_id,destination_rank,destination_id,proximity_traveltime,proximity_kilometers,SHAPE
0,996388782,1,523638237,4.910469,1.971235,"{""hasM"": true, ""paths"": [[[-122.7020867, 45.44..."
1,996388782,2,249961152,5.216235,3.173499,"{""hasM"": true, ""paths"": [[[-122.7020867, 45.44..."
2,996388782,3,403490348,5.944897,4.094377,"{""hasM"": true, ""paths"": [[[-122.7020867, 45.44..."
3,996388782,4,523661049,5.568102,4.136225,"{""hasM"": true, ""paths"": [[[-122.7020867, 45.44..."


In [79]:
nearest_dest_df = dest_df[dest_df[dest_id_fld].isin(nearest_existing_df['destination_id'].values)]
nearest_dest_df

,OBJECTID,LOCNUM,CONAME,STREET,CITY,STATE,STATE_NAME,ZIP,ZIP4,NAICS,SIC,SALESVOL,HDBRCH,ULTNUM,EMPNUM,FRNCOD,ISCODE,SQFTCODE,LOC_NAME,STATUS,SCORE,SOURCE,REMOVE,open_status,SHAPE
67,68,523638237,STARBUCKS,MONROE PKWY,LAKE OSWEGO,OR,Oregon,97035,8875,72251505,581228,764.0,2,303651459,16.0,4,,3,PointAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.70369999970802, ""y"": 45.43380000022..."
68,69,249961152,STARBUCKS,SW BARBUR BLVD,PORTLAND,OR,Oregon,97219,5903,72251505,581228,1075.0,2,303651459,20.0,4,,2,PointAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.72200000023662, ""y"": 45.45289999966..."
77,78,403490348,GAIA JAVA,SW BARBUR BLVD,PORTLAND,OR,Oregon,97219,4017,72251505,581228,269.0,,000000000,5.0,,,2,StreetAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.70910000033945, ""y"": 45.45919999956..."
79,80,523661049,STARBUCKS,SW TERWILLIGER BLVD,PORTLAND,OR,Oregon,97219,4564,72251505,581228,484.0,2,303651459,9.0,4,,2,PointAddress,M,100.0,INFOGROUP,0,None,"{""x"": -122.68460000027113, ""y"": 45.46290000001..."


In [80]:
near_map = nearest_existing_df.spatial.plot()
nearest_dest_df.spatial.plot(map_widget=near_map)
new_loc_df.spatial.plot(map_widget=near_map, symbol=symbol)
near_map.basemap = 'dark-gray-vector'
near_map

MapView(layout=Layout(height='400px', width='100%'))

## Select All Customer Origin Geographies Contributing to Any Nearest Locations

## Add New Location to Nearest Set

## Use Model to Re-Evaluate Retail Landscape